# Comparison with existing implemenation

Comparison with straight-forward existing implementation of IVAP algorithm  as implemented in https://github.com/ptocca/VennABERS

In [1]:
import numpy as np
from numpy import random
from sklearn.isotonic import IsotonicRegression
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import sys  
sys.path.insert(0, '/Users/ivanpetej/Projects/venn-abers-github/venn-abers/src/')

from venn_abers import VennAbers

### Define functions(from https://github.com/ptocca/VennABERS)

In [2]:
def push(x, stack):
    stack.append(x)


def pop(stack):
    return stack.pop()


def top(stack):
    return stack[-1]


def nextToTop(stack):
    return stack[-2]


# perhaps inefficient but clear implementation
def nonleftTurn(a, b, c):
    d1 = b - a
    d2 = c - b
    return np.cross(d1, d2) <= 0


def nonrightTurn(a, b, c):
    d1 = b - a
    d2 = c - b
    return np.cross(d1, d2) >= 0


def slope(a, b):
    ax, ay = a
    bx, by = b
    return (by - ay) / (bx - ax)


def notBelow(t, p1, p2):
    p1x, p1y = p1
    p2x, p2y = p2
    tx, ty = t
    m = (p2y - p1y) / (p2x - p1x)
    b = (p2x * p1y - p1x * p2y) / (p2x - p1x)
    return (ty >= tx * m + b)


kPrime = None


# Because we cannot have negative indices in Python (they have another meaning), I use a dictionary

def algorithm1(P):
    global kPrime

    S = []
    P[-1] = np.array((-1, -1))
    push(P[-1], S)
    push(P[0], S)
    for i in range(1, kPrime + 1):
        while len(S) > 1 and nonleftTurn(nextToTop(S), top(S), P[i]):
            pop(S)
        push(P[i], S)
    return S


def algorithm2(P, S):
    global kPrime

    Sprime = S[::-1]  # reverse the stack

    F1 = np.zeros((kPrime + 1,))
    for i in range(1, kPrime + 1):
        F1[i] = slope(top(Sprime), nextToTop(Sprime))
        P[i - 1] = P[i - 2] + P[i] - P[i - 1]
        if notBelow(P[i - 1], top(Sprime), nextToTop(Sprime)):
            continue
        pop(Sprime)
        while len(Sprime) > 1 and nonleftTurn(P[i - 1], top(Sprime), nextToTop(Sprime)):
            pop(Sprime)
        push(P[i - 1], Sprime)
    return F1


def algorithm3(P):
    global kPrime

    S = []
    push(P[kPrime + 1], S)
    push(P[kPrime], S)
    for i in range(kPrime - 1, 0 - 1, -1):  # k'-1,k'-2,...,0
        while len(S) > 1 and nonrightTurn(nextToTop(S), top(S), P[i]):
            pop(S)
        push(P[i], S)
    return S


def algorithm4(P, S):
    global kPrime

    Sprime = S[::-1]  # reverse the stack

    F0 = np.zeros((kPrime + 1,))
    for i in range(kPrime, 1 - 1, -1):  # k',k'-1,...,1
        F0[i] = slope(top(Sprime), nextToTop(Sprime))
        P[i] = P[i - 1] + P[i + 1] - P[i]
        if notBelow(P[i], top(Sprime), nextToTop(Sprime)):
            continue
        pop(Sprime)
        while len(Sprime) > 1 and nonrightTurn(P[i], top(Sprime), nextToTop(Sprime)):
            pop(Sprime)
        push(P[i], Sprime)
    return F0


def prepareData(calibrPoints):
    global kPrime

    ptsSorted = sorted(calibrPoints)

    xs = np.fromiter((p[0] for p in ptsSorted), float)
    ys = np.fromiter((p[1] for p in ptsSorted), float)
    ptsUnique, ptsIndex, ptsInverse, ptsCounts = np.unique(xs,
                                                           return_index=True,
                                                           return_counts=True,
                                                           return_inverse=True)
    a = np.zeros(ptsUnique.shape)
    np.add.at(a, ptsInverse, ys)
    # now a contains the sums of ys for each unique value of the objects

    w = ptsCounts
    yPrime = a / w
    yCsd = np.cumsum(w * yPrime)  # Might as well do just np.cumsum(a)
    xPrime = np.cumsum(w)
    kPrime = len(xPrime)

    return yPrime, yCsd, xPrime, ptsUnique


def computeF(xPrime, yCsd):
    global kPrime
    P = {0: np.array((0, 0))}
    P.update({i + 1: np.array((k, v)) for i, (k, v) in enumerate(zip(xPrime, yCsd))})

    S = algorithm1(P)
    F1 = algorithm2(P, S)

    P = {0: np.array((0, 0))}
    P.update({i + 1: np.array((k, v)) for i, (k, v) in enumerate(zip(xPrime, yCsd))})
    P[kPrime + 1] = P[kPrime] + np.array((1.0, 0.0))  # The paper says (1,1)

    S = algorithm3(P)
    F0 = algorithm4(P, S)

    return F0, F1


def getFVal(F0, F1, ptsUnique, testObjects):
    pos0 = np.searchsorted(ptsUnique, testObjects, side='left')
    pos1 = np.searchsorted(ptsUnique[:-1], testObjects, side='right') + 1
    return F0[pos0], F1[pos1]


def ScoresToMultiProbs(calibrPoints, testObjects):
    # sort the points, transform into unique objects, with weights and updated values
    yPrime, yCsd, xPrime, ptsUnique = prepareData(calibrPoints)

    # compute the F0 and F1 functions from the CSD
    F0, F1 = computeF(xPrime, yCsd)

    # compute the values for the given test objects
    p0, p1 = getFVal(F0, F1, ptsUnique, testObjects)

    return p0, p1

# manual VennABERS calculation applying isotonic regression
def VennABERS_by_def(ds, test):
    p0, p1 = [], []
    for x in test:
        ds0 = ds + [(x, 0)]
        iso0 = IsotonicRegression().fit(*zip(*ds0))
        p0.append(iso0.predict([x]))

        ds1 = ds + [(x, 1)]
        iso1 = IsotonicRegression().fit(*zip(*ds1))
        p1.append(iso1.predict([x]))
    return np.array(p0).flatten(), np.array(p1).flatten()

### Example dataset (from https://github.com/ptocca/VennABERS)

In [3]:
np.random.seed(0)

def sigmoid(x):
    return np.exp(-np.logaddexp(0,-x))

def thr(xs):
    return 0.5*(sigmoid((xs+4))+sigmoid(4*(xs-4)))

def classAssignment(xs):
    global thr
    u = np.random.random(size=xs.shape[0])
    ys = u<thr(xs)
    return ys

In [4]:
xs = np.random.uniform(low=-10,high=10,size=400)
ys = classAssignment(xs)

xtest = np.linspace(-11,11,1000)

**Existing implementation**

In [5]:
startTime = datetime.now()
p0_g, p1_g = ScoresToMultiProbs(list(zip(xs,ys)),xtest)
print('Time taken ' + str(datetime.now() - startTime))

Time taken 0:00:00.018618


Comparison with manual VennABERS calculation as outlined in "Venn-Abers predictors" by Vovk and Petej (https://arxiv.org/abs/1211.0025), we refer to this is "Our implementation"

In [6]:
p0_v, p1_v = VennABERS_by_def(list(zip(xs,ys)),xtest)

In [7]:
discrepancies_p0 = np.argwhere(~np.isclose(p0_g.flatten(),p0_v.flatten()))
discrepancies_p1 = np.argwhere(~np.isclose(p1_g.flatten(),p1_v.flatten()))

print("p0: there are", discrepancies_p0.shape[0], "discrepancies")
print("p1: there are", discrepancies_p1.shape[0], "discrepancies")

p0: there are 0 discrepancies
p1: there are 0 discrepancies


**Our implementation**

In [8]:
p_cal = np.transpose(np.vstack((1-xs, xs)))
y_cal = ys
p_test = np.transpose(np.vstack((1-xtest, xtest)))

startTime = datetime.now()
va = VennAbers()
va.fit(p_cal, y_cal)
p_prime, probs = va.predict_proba(p_test)
print('Time taken: ' + str(datetime.now() - startTime))

Time taken: 0:00:00.003469


In [9]:
discrepancies_p0 = np.argwhere(~np.isclose(probs[:,0].flatten(),p0_v.flatten()))
discrepancies_p1 = np.argwhere(~np.isclose(probs[:,1].flatten(),p1_v.flatten()))

print("p0: there are", discrepancies_p0.shape[0], "discrepancies")
print("p1: there are", discrepancies_p1.shape[0], "discrepancies")

p0: there are 0 discrepancies
p1: there are 0 discrepancies


### Our example dataset (small)

In [10]:
p_cal = np.zeros((1000, 2))
y_cal = np.zeros((1000, 1))
p_test= np.zeros((1000, 2))

p_cal[:, 1] = random.randint(10000, size=1000) / 10000
p_cal[:, 0] = 1 - p_cal[:, 1]

y_cal = (p_cal[:, 1] > 0.3).astype('int')


p_test[:, 1] = random.randint(10000, size=1000) / 10000
p_test[:, 0] = 1 - p_test[:, 1]

**Existing implementation code**

In [11]:
startTime = datetime.now()
p0_g, p1_g = ScoresToMultiProbs(list(zip(p_cal[:, 1], y_cal)), p_test[:,1])
print('Time taken: ' + str(datetime.now() - startTime))

Time taken: 0:00:00.036466


**Our code**

In [12]:
startTime = datetime.now()
va = VennAbers()
va.fit(p_cal, y_cal)
p_prime, probs = va.predict_proba(p_test)
print('Time taken: ' + str(datetime.now() - startTime))

Time taken: 0:00:00.008986


In [13]:
p0_v, p1_v = VennABERS_by_def(list(zip(p_cal[:,1], y_cal)), p_test[:, 1])

**Discrepanices in existing implementation**

In [14]:
discrepancies_p0 = np.argwhere(~np.isclose(p0_g.flatten(),p0_v.flatten()))
discrepancies_p1 = np.argwhere(~np.isclose(p1_g.flatten(),p1_v.flatten()))

print("p0: there are", discrepancies_p0.shape[0], "discrepancies")
print("p1: there are", discrepancies_p1.shape[0], "discrepancies")

p0: there are 24 discrepancies
p1: there are 37 discrepancies


**Discrepanices in our implementation**

In [15]:
discrepancies_p0 = np.argwhere(~np.isclose(probs[:,0].flatten(),p0_v.flatten()))
discrepancies_p1 = np.argwhere(~np.isclose(probs[:,1].flatten(),p1_v.flatten()))

print("p0: there are", discrepancies_p0.shape[0], "discrepancies")
print("p1: there are", discrepancies_p1.shape[0], "discrepancies")

p0: there are 0 discrepancies
p1: there are 0 discrepancies


### Our example dataset (large)

In [16]:
p_cal = np.zeros((100000, 2))
p_test = np.zeros((100000, 2))

p_cal[:, 1] = random.randint(10000, size=100000) / 10000
p_cal[:, 0] = 1 - p_cal[:, 1]
y_cal = (p_cal[:, 1] > 0.3).astype('int')


p_test[:, 1] = random.randint(10000, size=100000) / 10000
p_test[:, 0] = 1 - p_test[:, 1]

**Existing implementation code**

In [17]:
startTime = datetime.now()
p0_g, p1_g = ScoresToMultiProbs(list(zip(p_cal[:, 1], y_cal)), p_test[:,1])
print('Time taken: ' + str(datetime.now() - startTime))

Time taken: 0:00:00.401858


**Our code**

In [18]:
startTime = datetime.now()
va = VennAbers()
va.fit(p_cal, y_cal)
p_prime, probs = va.predict_proba(p_test)
print('Time taken: ' + str(datetime.now() - startTime))

Time taken: 0:00:00.216015


Our implmentaion yields a faster computation time